### Импорт библиотек

In [1]:
from bs4 import BeautifulSoup as bs

In [3]:
import pandas as pd

In [5]:
import requests

### Планы предстоящей работы:
Нужно вывести следующие показатели, и сохранить в датасет 
- Название
- Год выпуска
- Длительность
- Описание
- Жанр
- Страна

Проверить записи в нашем датасете

In [7]:
url = "https://www.imdb.com/chart/top/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)
soup = bs(response.text, 'html.parser')

result_list = []

script_tag = soup.find('script', {'id': '__NEXT_DATA__', 'type': 'application/json'})
data = json.loads(script_tag.string)

movies = data['props']['pageProps']['pageData']['chartTitles']['edges']

films_data = []

for movie in movies:
    node = movie['node']
    print(node)
    title = node['titleText']['text']
    year = node['releaseYear']['year']
    runtime = node['runtime']['seconds']
    genres = ", ".join([genre['genre']['text'] for genre in node['titleGenres']['genres']])
    url = 'https://www.imdb.com/title/' + node['id']
    page = requests.get(url, headers=headers)
    soup = bs(page.text, 'html.parser')
    country_tag = soup.find('li', {'data-testid': 'title-details-origin'})
    if country_tag:
        country_links = country_tag.find_all('a')
        country = ", ".join(link.text for link in country_links)
    else:
        country = "N/A"
    plot = node['plot']['plotText']['plainText']

    films_data.append({
        "Название": title,
        "Год выпуска": year,
        "Длительность": f"{runtime // 3600}h {(runtime % 3600) // 60}m",
        "Описание": plot,
        "Жанр": genres,
        "Страна": country
    })


{'id': 'tt0111161', 'titleText': {'text': 'Побег из Шоушенка', '__typename': 'TitleText'}, 'titleType': {'id': 'movie', 'text': 'Movie', 'canHaveEpisodes': False, 'displayableProperty': {'value': {'plainText': '', '__typename': 'Markdown'}, '__typename': 'DisplayableTitleTypeProperty'}, '__typename': 'TitleType'}, 'originalTitleText': {'text': 'The Shawshank Redemption', '__typename': 'TitleText'}, 'primaryImage': {'id': 'rm1690056449', 'width': 1200, 'height': 1800, 'url': 'https://m.media-amazon.com/images/M/MV5BMDAyY2FhYjctNDc5OS00MDNlLThiMGUtY2UxYWVkNGY2ZjljXkEyXkFqcGc@._V1_.jpg', 'caption': {'plainText': 'Tim Robbins in Побег из Шоушенка (1994)', '__typename': 'Markdown'}, '__typename': 'Image'}, 'releaseYear': {'year': 1994, 'endYear': None, '__typename': 'YearRange'}, 'ratingsSummary': {'aggregateRating': 9.3, 'voteCount': 3018944, '__typename': 'RatingsSummary'}, 'runtime': {'seconds': 8520, '__typename': 'Runtime'}, 'certificate': {'rating': '16+', '__typename': 'Certificate'}

### Создаем дата сет, для хранения информации о фильмах

In [9]:
df = pd.DataFrame(films_data)

In [17]:
df

,Название,Год выпуска,Длительность,Описание,Жанр,Страна
0,Побег из Шоушенка,1994,2h 22m,A banker convicted of uxoricide forms a friend...,Drama,United States
1,Крёстный отец,1972,2h 55m,The aging patriarch of an organized crime dyna...,"Crime, Drama",United States
2,Тёмный рыцарь,2008,2h 32m,When a menace known as the Joker wreaks havoc ...,"Action, Crime, Drama","United States, United Kingdom"
3,Крёстный отец 2,1974,3h 22m,The early life and career of Vito Corleone in ...,"Crime, Drama",United States
4,12 рaзгневанных мужчин,1957,1h 36m,The jury in a New York City murder trial is fr...,"Crime, Drama",United States
...,...,...,...,...,...,...
245,Форма голоса,2016,2h 10m,"A deaf girl, Shoko, is bullied by the popular ...","Animation, Drama",Japan
246,Прислуга,2011,2h 26m,An aspiring author during the civil rights mov...,Drama,"United States, India"
247,Сука любовь,2000,2h 34m,"An amateur dog fighter, a supermodel, and a de...","Drama, Thriller",Mexico
248,Ребекка,1940,2h 10m,A self-conscious woman juggles adjusting to he...,"Drama, Mystery, Romance",United States


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Название      250 non-null    object
 1   Год выпуска   250 non-null    int64 
 2   Длительность  250 non-null    object
 3   Описание      250 non-null    object
 4   Жанр          250 non-null    object
 5   Страна        250 non-null    object
dtypes: int64(1), object(5)
memory usage: 11.8+ KB


In [15]:
df.describe()

,Год выпуска
count,250.000000
mean,1988.432000
std,25.601153
min,1921.000000
25%,1972.250000
50%,1995.000000
75%,2009.000000
max,2024.000000
